In [1]:
!pip install -q openml


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms import Compose, ToTensor, Resize
from torch import optim
import numpy as np
import pandas as pd
from torch.hub import tqdm
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

load_dotenv()

## uncomment if you want to ignore warnings
import warnings
warnings.filterwarnings('ignore')

C:\Users\patri\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import OpenmlDataset, OpenmlDatasetLoader
from datasets import dataset_train_test_split
from preencoder import PreEncoder 
import numpy as np
from models import ToyModel          
### example task_ids ###

# 361066 bank-marketing
# 361076 wine_quality
# 361085 sulfur
# 361088 superconduct
# 361089 california
# 361110 electricity
# 361111 eye_movements
# 361112 KDDCup09_upselling
# 361114 rl
# 361116 compass
# 361099 Bike_Sharing_Demand
# 361102 house_sales

task_id =  361099 #361076

dataset_loader = OpenmlDatasetLoader()
dataset = dataset_loader.load(task_id)
dataset.print_info()


-------------------------------
Dataset name: Bike_Sharing_Demand
n_samples: 17379
m_features: 11 (including 5 categorical features)
Task: regression
-------------------------------


### Faire un classement de dataset exclusivement numeric / both types

In [4]:
dataset.X

,season,year,month,hour,holiday,workingday,weather,temp,feel_temp,humidity,windspeed
0,1,0,1,0,0,0,0,9.84,14.395,0.81,0.0000
1,1,0,1,1,0,0,0,9.02,13.635,0.80,0.0000
2,1,0,1,2,0,0,0,9.02,13.635,0.80,0.0000
3,1,0,1,3,0,0,0,9.84,14.395,0.75,0.0000
4,1,0,1,4,0,0,0,9.84,14.395,0.75,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
17374,1,1,12,19,0,1,2,10.66,12.880,0.60,11.0014
17375,1,1,12,20,0,1,2,10.66,12.880,0.60,11.0014
17376,1,1,12,21,0,1,0,10.66,12.880,0.60,11.0014
17377,1,1,12,22,0,1,0,10.66,13.635,0.56,8.9981


In [9]:
def softmax(X):
    res = []
    for x in X:
        exp = np.exp(x)
        res.append(exp / exp.sum())
    return res

def forward(model, X, return_cache=True):
    cache = {}
    
    cache["a1"] = X @ model["w1"]
    cache["a2"] = cache["a1"] @ model["w2"]
    cache["z"] = softmax(cache["a2"])
    
    if not return_cache:
        return cache["z"]
    return cache


def cross_entropy(z, y):
    return - np.sum(np.log(z) * y)

def backward(model, X, y, alpha):
    cache  = forward(model, X)
    da2 = cache["z"] - y
    dw2 = cache["a1"].T @ da2
    da1 = da2 @ model["w2"].T
    dw1 = X.T @ da1
    assert(dw2.shape == model["w2"].shape)
    assert(dw1.shape == model["w1"].shape)
    model["w1"] -= alpha * dw1
    model["w2"] -= alpha * dw2
    return cross_entropy(cache["z"], y)